# Interactive dynamic simulations: create your own mutants to study mRNA stability

Author: Anna B. Matuszyńska <a href="https://orcid.org/0000-0003-0882-6088"><img src="https://orcid.org/assets/vectors/orcid.logo.icon.svg" width=15></a>

In this notebook we explore the ideas presented in the original work by Hargrove and Schmidt [1] and White, Brewer and Wilson [2] on the effect that changing the rate of synthesis and degradation of mRNA would have on the final concentration. We play with the simplest, well established mathematical model based on a single compartment, with zero-order rate kinetics for mRNA synthesis and first-order rate of decay. 

This notebook aims at further supporting hypotheses generated by **Srimeenakshi Sankaranarayanan**, Carl Haag, Patrick Petzsch, Karl Köhrer, Kathi Zarnack and **Michael Feldbrügge** in their manuscript.

[1] Hargrove,J.L.; Schmidt, F. H. [The role of mRNA and protein stability in gene expression.](https://faseb.onlinelibrary.wiley.com/doi/10.1096/fasebj.3.12.2676679) *FASEB J*. 3: 2360-2370; 1989.

[2] White, E.J.F.; Brewer, G.; Wilson, G.M. [Post-transcriptional control of gene expression by AUF1:
Mechanisms, physiological targets, and regulation.](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3664190/) *Biochim Biophys Acta.* 1829(0): 680–688; 2013.

In [1]:
%matplotlib inline

import numpy as np
import pandas

import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt

from modelbase.ode import Model, Simulator, mca
from modelbase.ode import ratelaws as rl

In [2]:
def decay(halftime):
    """ return the rate of mRNA decay for the given half-life"""
    return np.log(2)/halftime

m = Model()

m.add_compounds(["mRNA"])
m.add_parameters({'kSynth': 1, 'kDecay': decay(0.0578)})

def constant(k):
    return k

def mass_action_1s(s, kp):
    """Irreversible mass-action function with 1 substrate
    Arguments:
    s -> Substrate
    kp -> positive rate constant 
    """
    return kp * s

m.add_reaction_from_args(
    rate_name = "vSynth", #This should be a unique name that helps you identify the appropriate reaction
    function = constant, #This is the appropriate function you created beforehand
    stoichiometry = {"mRNA": 1}, #This is a dictionary of the compounds with their respective stochiometry in this specific reaction
    args = ["kSynth"] #this is a list of all arguments passed to the function, in their respective oder
)

m.add_reaction_from_args(
    rate_name = "vDecay",
    function = mass_action_1s,
    stoichiometry = {"mRNA": -1},
    args = ["mRNA", "kDecay"],
    reversible = False
)

In [6]:
import ipywidgets as widgets
from IPython.display import display

widgets.interact_manual.opts['manual_name'] = 'Run simulation'

@widgets.interact_manual(
    y0= widgets.IntSlider(
    value=36,
    min=0,
    max=1000,
    step=1,
    description='[mRNA] at t0',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
), synthesis=widgets.IntSlider(
    value=36,
    min=0,
    max=1000,
    step=1,
    description='synthesis',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
), halftime=(1., 180.))
def plot(y0=36., synthesis=30, halftime=8, grid=True):
    s = Simulator(m)
    y0 = {'mRNA': y0}
    m.update_parameters({'kSynth': synthesis, 'kDecay': decay(halftime)})
    s.initialise(y0)
    t, y = s.simulate(4*60) #Actually simulating the model until the given time point. The specifc time steps are stored as an array in the variable t
                                #and the different concentrations in a nested array
    findhalf = min([i for i in range(len(y)) if y[i] >= 0.5 * y[-1]])
    
    plt.plot(t[findhalf], 
             y[findhalf], marker='o', color='r', markersize=5)
    
    plt.axhline(y=y[findhalf], xmin=0, xmax=1, color='r', linestyle = "--")
    plt.axvline(x=t[findhalf], color='r', linestyle = "--")
    
    plt.plot(t,y, label = str(k), color='r')
    plt.xlabel("Time [au]")
    plt.ylabel("Concentration [au]") #modelbase gives us the option to plot immediately through our Simulator() object,
                                                                        #but we could have also done it with the t and y variables appointed earlier.
        

interactive(children=(IntSlider(value=36, continuous_update=False, description='[mRNA] at t0', max=1000), IntS…